AP en utilisant 2 coco files pour le tracking performance
Il faut creer le fichier d'annotation apres prediction sous le bon format
Le fichier gt, doit etre bon c'est celui creer lors de la validation dataset

In [116]:
# import the librairies
import cv2, os
from torch.cuda import is_available

# Define the parameters for the model:
datasetFolder = r"C:\Users\faraboli\Desktop\BubbleID\BubbleIDGit\ProjetBubbleID\training\DATASETS\dataset_tip_png"
imagesfolder = os.path.join(datasetFolder, "val/")   # Define the path to the folder of images
videopath = os.path.join(datasetFolder, "valVideo.avi")   # Define the path to the avi video
savefolder="../training/Output/"   # Define the folder you want the data to save in
savefolder2 = "../training/Performances_tip/" 
extension="all_jpg"    # Define the extension you want all the saved data to have. This should be unique for each experiment
thres=0.5    # Define the threshold for what the model identifies as a bubble
modelweights = "..\\MODELS\\" + "Models_3classes_all" + "\\model_final.pth"
# modelweights=r"C:\Users\faraboli\Desktop\BubbleID\BubbleIDGit\ProjetBubbleID\training\Models_3classes_all\model_final.pth"     # Define the path to the saved model weights.
# device='cpu'   # Specify if running on "cpu" or "gpu"
device = "cuda" if is_available() else "cpu"
print(f"Used device : {device}")


validationCOCO_path = os.path.join(datasetFolder, "val.json")


Si la video n'existe pas on la creer

In [117]:

# Dossier contenant les images

fichiers = sorted([f for f in os.listdir(imagesfolder) if f.endswith(".png")])
N_frame = len(fichiers)


if not os.path.exists(videopath):
    image_exemple = cv2.imread(os.path.join(imagesfolder, fichiers[0])) # Lire la première image pour obtenir la taille
    hauteur, largeur, _ = image_exemple.shape

    # Créer l'objet vidéo
    fps = 5  # images par seconde
    videopath = os.path.join(datasetFolder, "valVideo.avi")
    codec = cv2.VideoWriter_fourcc(*'XVID')  # ou 'XVID' pour .avi
    video = cv2.VideoWriter(videopath, codec, fps, (largeur, hauteur))

    # Ajouter chaque image à la vidéo
    for fichier in fichiers:
        chemin = os.path.join(imagesfolder, fichier)
        image = cv2.imread(chemin)
        video.write(image)

    # Finaliser
    video.release()
    print(f"Vidéo créée : {videopath}")

imagesfolder = os.path.join(datasetFolder, "trim/")

# Prediction sur le validation set

In [118]:
import importlib
import BubbleID_My

# recharge le module pour prendre en compte les modifications
BubbleID_My = importlib.reload(BubbleID_My)

import BubbleID_My as BubbleID
print(BubbleID.__file__)

# Instantiating the class
test120=BubbleID.DataAnalysis(imagesfolder,videopath,savefolder,extension,modelweights,device)


In [119]:
test120.trimVideo(N_frames_extr=N_frame)

In [120]:
# Generate the data from the images:
#test120.set_scale_by_two_points(frame_idx=0, physical_mm=20.0)
mm = test120.set_scale_by_two_points(frame_idx=0, physical_mm=20.0, save=True)
print("mm/px =", mm)

In [121]:
test120.GenerateData(thres, save_rich=True, save_masks=False, save_contours=True, iou_thresh_tid=0.5)

[2025-10-22 19:00:08] INFO - checkpoint.py - [Checkpointer] Loading from ..\MODELS\Models_3classes_all\model_final.pth ...
100%|██████████| 56/56 [00:06<00:00,  8.59it/s]


# Creation du coco prediction
Il faut creer le fichier coco à partir des predictions faites qui soit compatible avec le coco du validation set.
On utilise contours_ .json et rich_ .csv

In [122]:
# On creer le coco avec les fichier deja existant rich et contour
import json, csv


contours_json_path = savefolder + 'contours_' + extension + '.json'
rich_csv_path = savefolder + 'rich_' + extension + '.csv'
outputCoco_path = os.path.join(savefolder2,"predictions_" + extension + ".json")

# Lire le fichier JSON d'entrée
with open(contours_json_path, 'r', encoding='utf-8') as f_json:
    contours = json.load(f_json)

# Lire le fichier rich.csv et ne selectionner que les colonne qui nous interressent
donnees_filtrees = []

with open(rich_csv_path, "r", encoding="utf-8") as f:
    lecteur = csv.DictReader(f)
    for ligne in lecteur:
        entree = {
            "frame": int(ligne["frame"]),
            "det_in_frame": int(ligne["det_in_frame"]),
            "x1": int(ligne["x1"]),
            "y1": int(ligne["y1"]),
            "x2": int(ligne["x2"]),
            "y2": int(ligne["y2"]),
            "score": float(ligne["score"]),
            "class_id": int(ligne["class_id"])
        }
        donnees_filtrees.append(entree)


if len(donnees_filtrees) != len(contours):
    raise("Le fichier contours et rich n'ont pas le meme nb de lignes")

outputFile = []
for idx, imgCode in enumerate(contours):
    # on verifie qu'on parle de la meme image
    i, d = map(int, imgCode.split("_"))
    if i != (donnees_filtrees[idx]["frame"]) or d != (donnees_filtrees[idx]["det_in_frame"]):
        print("WARNING: les 2 fichiers ne sont pas compatibles")
        print(i, donnees_filtrees[idx]["frame"], d, donnees_filtrees[idx]["det_in_frame"])
        continue
    
    width = (donnees_filtrees[idx]["x2"]) - (donnees_filtrees[idx]["x1"])
    heigth = (donnees_filtrees[idx]["y2"]) - (donnees_filtrees[idx]["y1"])
    flat_coords = [x for pt in contours[imgCode] for x in pt]
    prediction = {
        "image_id": donnees_filtrees[idx]["frame"] - 1 , #Dans ce fichier les idx commencent a 1 alors que dans l'autre ils commencent a 0
        "category_id": donnees_filtrees[idx]["class_id"],
        "segmentation": [flat_coords],
        "score": donnees_filtrees[idx]["score"],
        "bbox": [donnees_filtrees[idx]["x1"], donnees_filtrees[idx]["y1"], width, heigth ]
    }
    outputFile.append(prediction)
os.makedirs(savefolder2, exist_ok=True)   
# Écrire dans un fichier JSON
with open(outputCoco_path, "w", encoding="utf-8") as f:
    json.dump(outputFile, f, ensure_ascii=False, indent=2)



# Evaluation des performances

In [123]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import sys
import io

# --- Charger les fichiers ---
annType = 'segm'  # 'bbox' pour détection, 'segm' pour segmentation

coco_gt = COCO(validationCOCO_path)       # fichier COCO ground truth
coco_dt = coco_gt.loadRes(outputCoco_path)  # fichier prédictions

# --- Évaluation ---
coco_eval = COCOeval(coco_gt, coco_dt, annType)
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
# --- Capture de la sortie de summarize ---
buffer = io.StringIO()
sys.stdout = buffer  # Redirige stdout vers le buffer
coco_eval.summarize()
sys.stdout = sys.__stdout__  # Restaure stdout

# --- Sauvegarde dans un fichier texte ---
outputResult_path = os.path.join(savefolder2,"performance_" + extension + ".json")

with open(outputResult_path, "w", encoding="utf-8") as f:
    f.write(buffer.getvalue())